In [ ]:
#%%
# Images Classifier Demonstration



In [ ]:
#%%
# --- Environment Setting ---
# Tensorflow
# ! python --version
# ! conda update -n base conda -y
## ! conda create -n tensorflow-env python=3.6 -y
# ! conda create -n tf20 python=3.6 -y
## ! activate tensorflow-env
# ! activate tf20
# ! pip install "tensorflow>=1.15,<2.0"
# ! pip install "tensorflow-gpu>=1.15,<2.0"
# ! conda install tensorflow -y

# Nodejs
# ! curl -sL https://deb.nodesource.com/setup_12.x | bash -
# ! apt install nodejs -y

# Extension
# ! pip install sklearn
# ! pip install keras
# ! pip install --upgrade tensorflow-hub
# ! pip install keras==2.2.4
# ! pip list



In [ ]:
#%%
## Global



In [ ]:
#%%
# --- Libraries ---
# Debug
from IPython import embed
# Preparation
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Keras
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import Callback, TensorBoard

# --- Variables ---
class_label = ('cat', 'crow', 'horse', 'lion', 'giraffe')
image_size = 100
image_num = 100
# home_dir = '/data/notebooks/traffic/demo/'
home_dir = '/workspace/'
# image_dir = home_dir + 'images/'
image_dir = '/images/'
data_dir = home_dir + 'data/'
log_dir = home_dir + 'logs/'
X = []
Y = []
batch_size = 32
epochs = 20



In [ ]:
#%%
## Data Collection



In [ ]:
#%%
# --- Download ---
# ! git clone https://github.com/Joeclinton1/google-images-download.git gid-joeclinton
# ! python ./gid-joeclinton//google_images_download/google_images_download.py -f jpg -ct full-color -t photo -k cat -o /images/
# ! python ./gid-joeclinton//google_images_download/google_images_download.py -f jpg -ct full-color -t photo -k crow -o /images/
# ! python ./gid-joeclinton//google_images_download/google_images_download.py -f jpg -ct full-color -t photo -k horse -o /images/
# ! python ./gid-joeclinton//google_images_download/google_images_download.py -f jpg -ct full-color -t photo -k lion -o /images/
# ! python ./gid-joeclinton//google_images_download/google_images_download.py -f jpg -ct full-color -t photo -k giraffe -o /images/



In [ ]:
#%%
## Data Preparation



In [ ]:
#%%
# --- Images Display ---
# %matplotlib inline
#
# label_num = 0
# print('Label: ' + class_label[label_num])
# image_path = image_dir + class_label[label_num] + '/'
# image_file = glob.glob(image_path + '*.jpg')
# for i, files in enumerate(image_file):
# 	if i >= 10:
# 		break
# 	else:
# 		plt.figure(figsize=(5, 5))
# 		# plt.subplot(4, 5, i+1)
# 		plt.imshow(mpimg.imread(files), cmap='gray')



In [ ]:
#%%
# --- Images Conversion ---
for i, labels in enumerate(class_label):
	image_path = image_dir + labels + '/'
	image_file = glob.glob(image_path + '*.jpg')
	for j, files in enumerate(image_file):
		if j >= image_num:
			break
		else:
			images = Image.open(files)
			images = images.convert('RGB')
			images = images.resize((image_size, image_size))
			X.append(np.asarray(images))
			Y.append(i)

X = np.array(X)
Y = np.array(Y)

# --- Classify Training Data and Test Data ---
# Training : Test = 3 : 1
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y)
# Save Conversion Data
np.save(data_dir + 'images.npy', (X_train, X_test, Y_train, Y_test))



In [ ]:
#%%
## Dataset Conversion



In [ ]:
#%%
# --- Normalization & One Hot Vector ---
def conversion(X_train, X_test, Y_train, Y_test):
	X_train = X_train.astype('float') / 256
	X_test = X_test.astype('float') / 256
	Y_train = np_utils.to_categorical(Y_train, len(class_label))
	Y_test = np_utils.to_categorical(Y_test, len(class_label))

	return(X_train, X_test, Y_train, Y_test)



In [ ]:
#%%
## Model Bench Mark



In [ ]:
#%%
# --- Define Model ---
def cnn(X_train, Y_train, log_dir):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
	model.add(Activation('relu'))
	model.add(Conv2D(32, (3, 3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))

	model.add(Conv2D(64, (3, 3), padding='same'))
	model.add(Activation('relu'))
	model.add(Conv2D(64, (3, 3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))

	model.add(Flatten())
	model.add(Dense(512))
	model.add(Activation('relu'))
	model.add(Dropout(0.5))
	model.add(Dense(5))
	model.add(Activation('softmax'))

	Optimizer = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

	model.compile(
		loss = 'categorical_crossentropy',
		optimizer = Optimizer,
		metrics = ['accuracy']
	)

	model.summary()

	model.fit(
		X_train, Y_train,
		batch_size = batch_size,
		epochs = epochs
	)

	model.save(data_dir + 'model.h5')

	return model

# --- Training ---
def main():
	# If above numpy 1.16.3, change allow_pickle=False
	# X_train, X_test, Y_train, Y_test = np.load(data_dir + 'images.npy')
	X_train, X_test, Y_train, Y_test = np.load(data_dir + 'images.npy', allow_pickle=True)
	X_train, X_test, Y_train, Y_test = conversion(X_train, X_test, Y_train, Y_test)
	model = cnn(X_train, Y_train, log_dir)

if __name__ == "__main__":
	main()



In [ ]:
#%%
# --- Evaluate ---
def eval_model(X_test, Y_test, model):
	score = model.evaluate(X_test, Y_test, verbose=1)
	print('Loss: ', score[0])
	print('Accuracy: ', score[1])

def main():
	# If above numpy 1.16.3, change allow_pickle=False
	# X_train, X_test, Y_train, Y_test = np.load(data_dir + 'images.npy')
	X_train, X_test, Y_train, Y_test = np.load(data_dir + 'images.npy', allow_pickle=True)
	X_train, X_test, Y_train, Y_test = conversion(X_train, X_test, Y_train, Y_test)
	model = keras.models.load_model(data_dir + 'model.h5')
	eval_model(X_test, Y_test, model)

if __name__ == '__main__':
	main()



In [ ]:
#%%
# --- Predict ---
def main():
	model = keras.models.load_model(data_dir + 'model.h5')
	predict_label = 'cat'
	image_path = image_dir + predict_label + '/'
	image_file = glob.glob(image_path + '*.jpg')
	input_image = Image.open(image_file[9])
	input_image = input_image.convert('RGB')
	input_image = input_image.resize((image_size, image_size))
	input_image = np.asarray(input_image)

	X = []
	X.append(input_image)
	X = np.array(X)

	results = model.predict([X])[0]
	result = int(results[results.argmax()] * 100)
	print(image_file[9], ': {0}'.format(class_label[results.argmax()], result))

if __name__ == '__main__':
	main()